In [14]:
!bq load --source_format NEWLINE_DELIMITED_JSON --autodetect \
--ignore_unknown_values sourabhjainceanalytics:resulticks.target_table gs://bigquery_json_processing/target_table.json

Waiting on bqjob_r50fbccd557eef730_0000017e28d28b58_1 ... (1s) Current status: DONE   


In [15]:
%%bigquery
SELECT * FROM `sourabhjainceanalytics.resulticks.target_table`

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.31rows/s]


,region,name,salary,id
0,Mumbai,Sourabh,100,1
1,Mumbai,Rajesh,200,2
2,Mumbai,Nishit,300,3
3,Mumbai,Rakesh,400,4
4,Mumbai,Vivek,500,5


In [16]:
!bq load --source_format NEWLINE_DELIMITED_JSON --autodetect \
--ignore_unknown_values sourabhjainceanalytics:resulticks.change_table gs://bigquery_json_processing/change_table.json

Waiting on bqjob_r34f86ac7ea80566d_0000017e28d2a080_1 ... (1s) Current status: DONE   


In [17]:
%%bigquery
SELECT * FROM `sourabhjainceanalytics.resulticks.change_table`

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.34rows/s]


,region,salary,id
0,None,1000.0,1
1,Mumbai,NaN,2
2,Chennai,3000.0,3


In [19]:
%%bigquery
SELECT * FROM `sourabhjainceanalytics.resulticks.target_table`

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.71rows/s]


,region,name,salary,id
0,Chennai,Nishit,3000,3
1,Mumbai,Vivek,500,5
2,Mumbai,Rakesh,400,4
3,Mumbai,Rajesh,200,2
4,Mumbai,Sourabh,1000,1


In [ ]:
Now we have seen that the value are updated appropriately. However we will not be sure of the columns that would come in the JSON file and hence we need to make 
the below clause dynamic based on the attributes in the JSON file.

 T.region = CASE WHEN S.region IS NULL THEN T.region ELSE S.region END
,T.salary = CASE WHEN S.salary IS NULL THEN T.salary ELSE S.salary END

BigQuery provides metadata about the tables and we can leverage that to find the columns. In the below query we provide name of the table for which we want to find the columns
and also ignore the key column in the output. We can use this information and write a custom script to prepare the clause dynamically.

In [21]:
%%bigquery
SELECT column_name 
FROM sourabhjainceanalytics.resulticks.INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('change_table') 
and column_name NOT IN ('id')

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.26rows/s]


,column_name
0,region
1,salary
